In [1]:
%run scripts/python/standardize.py
%matplotlib inline
pd.set_option('max_columns',50)

In [2]:
sd_data = pd.read_csv('_data/blds_csvs/cities/San_Diego.csv')

In [5]:
add_year_month_day_df(sd_data,'issueddate','issued')

In [6]:
sd_data.head(1)

,Unnamed: 0,addedsqft,applieddate,coissueddate,completeddate,contractoraddress1,contractoraddress2,contractorcity,contractorcompanydesc,contractorcompanyname,contractoremail,contractorfullname,contractorlicnum,contractorphone,contractorstate,contractorstatelic,contractortrade,contractortrademapped,contractorzip,description,estprojectcost,expiresdate,fee,housingunits,issueddate,...,link,masterpermitnum,originaladdress1,city,state,originalzip,pin,permitclass,permitclassmapped,permitnum,permittype,permittypedesc,permittypemapped,proposedused,publisher,status,statuscurrentmapped,statusdate,totalsqft,workclass,workclassmapped,id,issuedYear,issuedMonth,issuedDay
0,0,NaN,2017-07-13,NaN,NaN,1800 W ASHTON BOULEVARD,500,LEHI,NaN,VIVINT SOLAR DEVELOPER LLC,na,Cody Oram,973756.0,855 877-2974,UT,CA,CSLB Contractor,NaN,84043.0,CROSS STREET: 14th ST \r\n\r\nDescription of W...,NaN,NaN,0.0,0.0,2017-07-13,...,https://publicservices.sdcounty.ca.gov/citizen...,NaN,1395 LA HAINA ST,RAMONA,CA,92065.0,NaN,Residential Alteration-Addn,Residential,PDS2017-RESALT-005662,Residential Alteration or Addition - Plan Chec...,8002 - REN - Solar Photovoltaic Roof Mount Res...,NaN,CROSS STREET: 14th ST \n\nDescription of Work:...,San Diego - Prod 7.3.3.8.1,Issued,Permit Issued,2017-07-13,NaN,Existing,Existing,81266,2017.0,7.0,3.0


In [7]:
sd_data.to_csv('_data/San_Diego.csv')

In [7]:
sd_plot = sd_data.groupby(['permittypemapped','issuedMonth']).size().to_frame().reset_index().rename(columns={0:'tally'})

In [5]:
sd_data.groupby(['permittypemapped','issuedYear']).size().to_frame().reset_index()

,permittypemapped,issuedYear,0
0,Building,2017.0,8
1,Electrical,2010.0,1
2,Electrical,2015.0,3
3,Electrical,2017.0,40
4,Mechanical,2015.0,2
5,Mechanical,2017.0,14
6,Other,2010.0,1
7,Other,2017.0,6
8,Plumbing,2017.0,12
9,Pool/Spa,2017.0,1


This is clearly not a comprehensive dataset. I wonder why San Diego is only able to provide data for 3 years (2010,15, and 17) after 2010.

In [8]:
sd_plot.to_csv('_data/sd_plot.csv')

In [57]:
sd_by_permittype = {}
for permittype in set(sd_plot.permittypemapped.values):
    if permittype == 'Pool/Spa':
        permittype = 'Pool_Spa'
    sd_by_permittype[permittype] = sd_plot[sd_plot.permittypemapped == permittype]

In [ ]:
sd_by_permittype = {}
for permittype in set(sd_plot.permittypemapped.values):
    if permittype == 'Pool/Spa':
        permittype = 'Pool_Spa'
    sd_plot[sd_plot.permittypemapped == permittype].to_csv('_data/SD_'+permittype+'.csv')

In [41]:
sd_plot[sd_plot.issuedMonth == 5]

,permittypemapped,issuedMonth,tally
1,Building,5.0,7
4,Electrical,5.0,9
9,Mechanical,5.0,3
16,Plumbing,5.0,1


In [37]:
sd_plot

,permittypemapped,issuedMonth,tally
0,Building,3.0,1
1,Building,5.0,7
2,Electrical,2.0,1
3,Electrical,3.0,30
4,Electrical,5.0,9
5,Electrical,6.0,1
6,Electrical,8.0,2
7,Electrical,9.0,1
8,Mechanical,3.0,7
9,Mechanical,5.0,3


In [128]:
#df is the permittype specific df
def make_tally_list(df,permittype,permit_col, month_col):
    tallies = [0]*12
    tallies.insert(0,permittype)
    df_by_permittype= df[df[permit_col]==permittype]
    for m in range(1,13):
        if m in set(df_by_permittype['issuedMonth'].values):
            tallies[m] = int(df_by_permittype[df_by_permittype['issuedMonth'] == m].tally)
    return tallies

In [129]:
def tallies_by_permit(df,permit_col, month_col):
    tally_permit_list = []
    for p_type in set(df[permit_col].values):
        print(p_type)
        tally_permit_list.append(make_tally_list(df, p_type, permit_col,month_col))
    return tally_permit_list

In [130]:
sd_tallies = tallies_by_permit(sd_plot,'permittypemapped','issuedMonth')

Pool/Spa
Other
Mechanical
Building
Electrical
Plumbing


In [137]:
sd_tallies

[['Pool/Spa', 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 ['Other', 0, 0, 5, 1, 0, 0, 0, 0, 0, 0, 0, 1],
 ['Mechanical', 0, 0, 7, 0, 3, 0, 4, 2, 0, 0, 0, 0],
 ['Building', 0, 0, 1, 0, 7, 0, 0, 0, 0, 0, 0, 0],
 ['Electrical', 0, 1, 30, 0, 9, 1, 0, 2, 1, 0, 0, 0],
 ['Plumbing', 0, 0, 8, 0, 1, 0, 3, 0, 0, 0, 0, 0]]

In [141]:
pd.DataFrame(sd_tallies).to_csv('_data/sd_tallies.csv')